In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from string import punctuation
from gensim.models import Doc2Vec
from collections import namedtuple
from random import shuffle
import logging
import numpy as np

# Reading Text file , extracting Notes content
def getProcessedText(data):
    return str(data).decode('ascii','ignore').encode('ascii','ignore')

dataframe = pd.read_csv("notesOutput.txt", sep=' ',index_col=None)
all_notes = [getProcessedText(doc) for doc in dataframe["Notes"]]
all_Outcomes = [outcome for outcome in dataframe["Autoclose"]]

_stop_words = set(stopwords.words('english') + list(punctuation))

#logging.basicConfig(format='%(asctime)s : %(message)s  : %(levelname)', level=logging.INFO)

def preprocessed_notes():
    for index , note in enumerate(all_notes):
        stop_word_free_note = ' '.join([w for w in note.split() if w not in _stop_words])
        yield gensim.utils.simple_preprocess(stop_word_free_note)

def sentences_perm(data):
    shuffle(data)
    return data


processedNotes = list(preprocessed_notes())
#print processedNotes

documents =[]
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for index , wordList in enumerate(processedNotes):
        tag_prefix = ""
        if all_Outcomes[index] == 'Positive':
            tag_prefix = "pos"
        else:
            tag_prefix = "neg"
        documents.append(analyzedDocument(wordList, [tag_prefix +'_' + str(index)]))
        
#print documents
    
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)

model.build_vocab(documents)

model.train(sentences_perm(documents), total_examples= len(documents), epochs=10)

model.save('./review.d2v')

model = Doc2Vec.load('./review.d2v')

print model.wv.most_similar('interest')

# Divinding data into Train-test data in 70:30
X = documents
y =[]
for index, doc in enumerate(documents):
    y.append(doc.tags[0].split('_')[0])


X_train , X_test, y_train , y_test = train_test_split(documents,y, test_size=0.3, random_state=21, stratify=y)

#classifier = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

train_array =[]
test_array =[]

for i , data in enumerate(X_train):
    train_array.append(model[data[1]][0])

for i , data in enumerate(X_test):
    test_array.append(model[data[1]][0])
    
classifier.fit(train_array, y_train)
classifier.score(test_array, y_test)



